In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from tensorflow import keras as keras
from matplotlib import pyplot as plt

from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Activation, Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Reshape, Concatenate, concatenate

### 数据处理

In [35]:
# 读取数据
data_ini = pd.read_excel('data/TJ POLTS.xlsx')

data = data_ini.iloc[:,[1,3,4,5,6,7,8]]
data.head()

,AQI,PM2.5,PM10,NO2,SO2,CO,O3_8h
0,310,260,420,139,201,3.5,13
1,225,175,271,111,143,3.2,31
2,275,225,360,128,197,3.9,12
3,126,96,183,70,101,2.3,26
4,116,88,159,70,114,1.6,16


In [36]:
# 指数平滑
def expPreprocessing(df, alpha=0.05):
    edata = df.ewm(alpha=alpha, adjust=False).mean()
    return edata

alpha = 0.05
data_exp = expPreprocessing(data, alpha)
data_exp.head()

# alpha = 0.05
# data_exp = data.ewm(alpha=alpha, adjust=False).mean()
# data_exp.head()

,AQI,PM2.5,PM10,NO2,SO2,CO,O3_8h
0,310.000000,260.000000,420.000000,139.0000,201.000000,3.500000,13.000000
1,305.750000,255.750000,412.550000,137.6000,198.100000,3.485000,13.900000
2,304.212500,254.212500,409.922500,137.1200,198.045000,3.505750,13.805000
3,295.301875,246.301875,398.576375,133.7640,193.192750,3.445462,14.414750
4,286.336781,238.386781,386.597556,130.5758,189.233112,3.353189,14.494013


In [37]:
# 归一化
min_value = data_exp.min(axis=0)  
max_value = data_exp.max(axis=0)

data_std = (data_exp - min_value) / (max_value - min_value)

In [38]:
# 划分训练集与验证集

time_stamp = 50  # 时间点长度
ratio = 0.8

split = int(ratio*len(data))
train_data = data_std[0:split + time_stamp]
valid_data = data_std[split - time_stamp:]

In [39]:
label_column = 0 # 标签所在的列id
forecast_horizon = 5 # 预测的步数
x_train, y_train = [], []
scaled_data = train_data.values
# 训练集
for i in range(time_stamp, len(train_data) - forecast_horizon + 1):
    x_train.append(scaled_data[i - time_stamp:i])
    y_train.append(scaled_data[i:i + forecast_horizon, label_column])

x_train, y_train = np.array(x_train), np.array(y_train)

# 验证集
x_valid, y_valid = [], []
for i in range(time_stamp, len(valid_data) - forecast_horizon + 1):
    x_valid.append(scaled_data[i - time_stamp:i])
    y_valid.append(scaled_data[i:i + forecast_horizon, label_column])

x_valid, y_valid = np.array(x_valid), np.array(y_valid)


### 贝叶斯参数优化

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import optuna
def InformerBlock(input_layer, head_size, num_heads, ff_dim, dropout_rate):
    # 概率稀疏自注意力层 (这里简化为普通的多头注意力)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout_rate)(input_layer, input_layer)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output + input_layer)

    # 前馈网络
    ffn_output = Dense(ff_dim, activation="relu")(attention_output)
    ffn_output = Dense(input_layer.shape[-1])(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    ffn_output = LayerNormalization(epsilon=1e-6)(ffn_output + attention_output)

    return ffn_output

def informer_model(input_shape, head_size, num_heads, ff_dim, num_blocks, dropout_rate):
    inputs = Input(shape=input_shape)
    x = inputs

    # Informer块
    for _ in range(num_blocks):
        x = InformerBlock(x, head_size, num_heads, ff_dim, dropout_rate)

    # 输出层
    x = Flatten()(x)
    outputs = Dense(5)(x)  # 假设目标是预测一个数值
    return Model(inputs, outputs)

class Trainer:
    def __init__(self, x_train, y_train, x_valid, y_valid, epochs=50, batch_size=64, n_trials=100):
        self.x_train, self.y_train, self.x_valid, self.y_valid = x_train, y_train, x_valid, y_valid
        self.epochs = epochs
        self.batch_size = batch_size
        self.n_trials = n_trials
    def objective(self, trial):
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
        head_size = trial.suggest_categorical('head_size', [32, 64, 128])
        num_heads = trial.suggest_int('num_heads', 1, 4)
        ff_dim = trial.suggest_int('ff_dim', 32, 128)
        num_blocks = trial.suggest_int('num_blocks', 1, 4)
        dropout_rate = trial.suggest_uniform('dropout_rate', 0., 0.5)

        model = informer_model(self.x_train.shape[1:], head_size, num_heads, ff_dim, num_blocks, dropout_rate)
        optimizer = Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        model.fit(self.x_train, self.y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0)

        y_pred = model.predict(self.x_valid)
        score = mean_squared_error(self.y_valid, y_pred)

        return score

    def optimizer_optuna(self):
        sampler = optuna.samplers.TPESampler()
        study = optuna.create_study(sampler=sampler, direction='minimize')
        study.optimize(self.objective, n_trials=self.n_trials, show_progress_bar=True)
        return study

In [41]:
# 超参数
epochs = 5
batch_size = 64
num_heads = 4
n_trials = 5
T = Trainer(x_train, y_train, x_valid, y_valid, epochs=epochs, batch_size=batch_size, n_trials=n_trials)
study = T.optimizer_optuna()
print(study.best_params)
print(study.best_value)

[I 2024-01-30 11:06:05,529] A new study created in memory with name: no-name-15c2d765-e934-40e7-bcb2-6e3b5c3bc098
D:\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


[I 2024-01-30 11:06:08,202] Trial 0 finished with value: 0.4472782776271231 and parameters: {'lr': 3.3162357056976886e-05, 'head_size': 32, 'num_heads': 2, 'ff_dim': 45, 'num_blocks': 1, 'dropout_rate': 0.11355969466440374}. Best is trial 0 with value: 0.4472782776271231.
[I 2024-01-30 11:06:20,967] Trial 1 finished with value: 0.00905340039938605 and parameters: {'lr': 0.00030984936682758166, 'head_size': 128, 'num_heads': 2, 'ff_dim': 73, 'num_blocks': 2, 'dropout_rate': 0.1880432425151407}. Best is trial 1 with value: 0.00905340039938605.
[I 2024-01-30 11:06:26,350] Trial 2 finished with value: 0.005734301904039597 and parameters: {'lr': 0.005446104238540606, 'head_size': 32, 'num_heads': 2, 'ff_dim': 112, 'num_blocks': 2, 'dropout_rate': 0.09224786261827478}. Best is trial 2 with value: 0.005734301904039597.
[I 2024-01-30 11:06:49,858] Trial 3 finished with value: 0.00523601504368883 and parameters: {'lr': 0.0003265072291503644, 'head_size': 64, 'num_heads': 3, 'ff_dim': 83, 'num_b

In [42]:
# 根据最优超参，训练和预测
params = study.best_params
lr = params['lr']
head_size = params['head_size']
num_heads = params['num_heads']
ff_dim = params['ff_dim']
num_blocks = params['num_blocks']
dropout_rate = params['dropout_rate']
checkpoint_path_best = "data/best.hdf5"
modelcheckpoint_best = keras.callbacks.ModelCheckpoint(checkpoint_path_best,
                                                       monitor='loss',
                                                       save_best_only=True,
                                                       mode='min',
                                                       verbose=0)
#LSTM

model = informer_model(x_train.shape[1:], head_size, num_heads, ff_dim, num_blocks, dropout_rate)

#print(model.summary())
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=lr))
model.fit(x_train, y_train, epochs=50, batch_size=batch_size, verbose=1, callbacks=[modelcheckpoint_best])

model.load_weights(checkpoint_path_best)
closing_price = model.predict(x_valid)

Epoch 1/50
41/41 [==============================] - 6s 109ms/step - loss: 0.1981
Epoch 2/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0190
Epoch 3/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0146
Epoch 4/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0128
Epoch 5/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0110
Epoch 6/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0096
Epoch 7/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0088
Epoch 8/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0081
Epoch 9/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0073
Epoch 10/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0068
Epoch 11/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0063
Epoch 12/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0059
Epoch 13/50
41/41 [======

In [43]:
import numpy as np
import pandas as pd

# 假设 max_value, min_value, alpha, label_column 已正确定义
# max_value = {...}
# min_value = {...}
# alpha = ...
# label_column = ...
y_valid_list = [y_valid]
# 反归一化
def unnormalize(data, max_value, min_value, label_column):
    return (data * (max_value[label_column] - min_value[label_column])) + min_value[label_column]

y_valid_original = unnormalize(pd.DataFrame(y_valid_list[0]), max_value, min_value, label_column)
predictions_original = unnormalize(pd.DataFrame(closing_price), max_value, min_value, label_column)

# 反平滑
# def exp_reversed(df, alpha):
#     reversed_df = pd.DataFrame()
#     for column in df.columns:
#         df_col = df[column]
#         row_0 = df_col.iloc[0]
#         df_t_1 = pd.Series([row_0]).append(df_col[:-1]).reset_index(drop=True)
#         result = (df_col - (1 - alpha) * df_t_1) / alpha
#         result.iloc[0] = df_col.iloc[0]
#         reversed_df = pd.concat([reversed_df, result], axis=1)
#     return reversed_df
original_y_valid = y_valid_original
original_predictions = predictions_original
# original_y_valid = exp_reversed(y_valid_original, alpha)
# original_predictions = exp_reversed(predictions_original, alpha)
print(original_y_valid)
print(original_predictions)
# 评估每一步的预测
metrics = {'RMSE': [], 'MAE': [], 'R2Score': [], 'MAPE': []}
for step in range(5):
    # 计算每个指标
    rmse = np.sqrt(mean_squared_error(original_y_valid.iloc[:, step], original_predictions.iloc[:, step]))
    mae = mean_absolute_error(original_y_valid.iloc[:, step], original_predictions.iloc[:, step])
    r2 = r2_score(original_y_valid.iloc[:, step], original_predictions.iloc[:, step])
    mape = np.mean(np.abs((original_y_valid.iloc[:, step] - original_predictions.iloc[:, step]) / original_y_valid.iloc[:, step])) * 100

    # 存储指标
    metrics['RMSE'].append(rmse)
    metrics['MAE'].append(mae)
    metrics['R2Score'].append(r2)
    metrics['MAPE'].append(mape)

# 输出指标
print(metrics)

              0           1           2           3           4
0    167.431957  166.060360  165.857342  169.814475  165.873751
1    166.060360  165.857342  169.814475  165.873751  160.430063
2    165.857342  169.814475  165.873751  160.430063  159.258560
3    169.814475  165.873751  160.430063  159.258560  155.195632
4    165.873751  160.430063  159.258560  155.195632  152.735850
..          ...         ...         ...         ...         ...
646   79.746129   78.708823   79.773382   85.684712   90.000477
647   78.708823   79.773382   85.684712   90.000477   92.700453
648   79.773382   85.684712   90.000477   92.700453   95.865430
649   85.684712   90.000477   92.700453   95.865430   96.222159
650   90.000477   92.700453   95.865430   96.222159   93.211051

[651 rows x 5 columns]
              0           1           2           3           4
0    155.696838  151.284653  150.699341  154.655518  149.746964
1    155.573257  151.235214  149.736389  153.486847  148.611938
2    154.557800 

In [44]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_valid_list = [y_valid]
# # 假设您的模型已经产生了5步预测
# predictions = model.predict(y_valid_list)  # 这应该是一个形状为 (n_samples, 5) 的数组

# 初始化指标
metrics = {'RMSE': [], 'MAE': [], 'R2Score': [], 'MAPE': []}
y_valid = pd.DataFrame(y_valid_list[0])
predictions =  pd.DataFrame(closing_price)
# 对于每一步预测
for step in range(5):
    # 计算每个指标
    rmse = np.sqrt(mean_squared_error(y_valid.iloc[:, step], predictions.iloc[:, step]))
    mae = mean_absolute_error(y_valid.iloc[:, step], predictions.iloc[:, step])
    r2 = r2_score(y_valid.iloc[:, step], predictions.iloc[:, step])
    mape = np.mean(np.abs((y_valid.iloc[:, step] - predictions.iloc[:, step]) / y_valid.iloc[:, step])) * 100

    # 存储指标
    metrics['RMSE'].append(rmse)
    metrics['MAE'].append(mae)
    metrics['R2Score'].append(r2)
    metrics['MAPE'].append(mape)

# 输出指标
print(metrics)


{'RMSE': [0.044263784142620236, 0.04701018682697216, 0.047417402918999436, 0.04625822775190598, 0.05102184528293646], 'MAE': [0.034689985121327135, 0.03806838072763376, 0.038365200453581445, 0.037389277650160924, 0.04062915888912315], 'R2Score': [0.7983880933321863, 0.7709215642093721, 0.7652311047025887, 0.774921566708515, 0.7238750086008228], 'MAPE': [21.244430468340543, 24.563982636611993, 23.671209132924922, 25.798067915096574, 26.225328149197896]}
